In [ ]:
!rm -rf output_report/

In [ ]:
from pyspark.sql import SparkSession

import os

from pyspark.sql import functions as f
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import collect_set, to_json, struct
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, DoubleType

trips_schema = StructType([
    StructField('vendor_id', StringType(), True),
    StructField('tpep_pickup_datetime', TimestampType(), True),
    StructField('tpep_dropoff_datetime', TimestampType(), True),
    StructField('passenger_count', IntegerType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('ratecode_id', IntegerType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('pulocation_id', IntegerType(), True),
    StructField('dolocation_id', IntegerType(), True),
    StructField('payment_type', IntegerType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('congestion_surcharge', DoubleType()),
])


def foreach_batch_function(df: DataFrame, epoch_id): # функция для записи батчей
    df.write.mode("append").json("output_report") # датафрейм будет писаться в папку output_report методом append


def main(spark: SparkSession):
    jsonOptions = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"} # указываем опции json-сериализации

    fields = list(map(lambda x: f"json_message.{x.name}", trips_schema.fields)) 

    df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "localhost:9092") \
        .option("subscribe", "taxi") \
        .option("startingOffsets", "latest") \
        .option("maxOffsetsPerTrigger", 1000) \
        .load() \
        .select(f.from_json(f.col("value").cast("string"), trips_schema, jsonOptions).alias("json_message")) \
        .select(fields)

    # пишем на диск
    writer = df \
        .writeStream \
        .foreachBatch(foreach_batch_function) \
        .trigger(processingTime='10 seconds') \
        .option("path", "output_report") \
        .outputMode("append") \
        .start()

    writer.awaitTermination()

In [ ]:
main(
    SparkSession \
    .builder \
    .config("spark.driver.host", "localhost")\
    .appName("streaming_job") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \
    .config("spark.driver.bindAddress","localhost") \
    .config("spark.ui.port","4050") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()
)